In [4]:
%pip install rasterio
%pip install pycountry
%pip install --upgrade holidays
%pip install pycountry_convert
%pip install timezonefinder

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 35.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.4 MB/s eta 0:00:00
  Created wheel for timezonefinder: filename=timezonefinder-6.2.0-cp310-cp310-manylinux_2_35_x86_64.whl size=46894970 sha256=2c02a2532cdaf31d70fb4acf095d54ce908b7ef72639ec67eab18f6869529be3
  Stored in directory: /home/jovyan/shared/anaconda/pip/wheels/27/df/c8/0e0c1bf5602ba93be355154b621ae5ac8c3404a729ffda4f36
Successfully built timezonefinder
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd 
import os
import requests
import matplotlib.pyplot as plt 
import rasterio
from datetime import date
import pycountry
import holidays
from rasterio.warp import transform
import pycountry_convert as pc
from timezonefinder import TimezoneFinder

In [3]:
lat = 49.083333
lon = -122.350
def get_location_data(lat, lon):
    url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&accept-language=en'
    try:
        res = requests.get(url)
        # Raise an exception if the response status is not 200 (HTTP_OK)
        res.raise_for_status()
    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return None
    except requests.exceptions.RequestException as err:
        print(f'Error occurred: {err}')
        return None

    # No error was raised, so we can return the JSON data
    return res.json()

data= get_location_data(lat, lon)
data

{'place_id': 300075813,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 1064266207,
 'lat': '49.078078',
 'lon': '-122.3508038',
 'display_name': 'Abbotsford, Fraser Valley Regional District, British Columbia, V4X 2M8, Canada',
 'address': {'city': 'Abbotsford',
  'county': 'Fraser Valley Regional District',
  'state': 'British Columbia',
  'ISO3166-2-lvl4': 'CA-BC',
  'postcode': 'V4X 2M8',
  'country': 'Canada',
  'country_code': 'ca'},
 'boundingbox': ['49.0745666', '49.078078', '-122.3512118', '-122.3496933']}

In [9]:
tf = TimezoneFinder()


In [10]:

tz = tf.timezone_at(lng=-122.3508038, lat=49.078078)
tz

TypeError: TimezoneFinder.timezone_at() got an unexpected keyword argument 'lon'

In [22]:
data["country"]

'Switzerland'

# GDP PPP in US dollars(2017)
https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.KD


In [26]:
gpd_PPP =  pd.read_csv("./Energy_graph/data/metadata/Wages/gdp-per-capita-worldbank.csv")
df = gpd_PPP[gpd_PPP["Year"] == 2020].drop(columns=["Year", "Code"])

df.reset_index(inplace=True, drop=True)
df.set_index("Entity", inplace=True)

In [27]:
gpd_PPP

,Entity,Code,Year,"GDP per capita, PPP (constant 2017 international $)"
0,Afghanistan,AFG,2002,1280.4631
1,Afghanistan,AFG,2003,1292.3335
2,Afghanistan,AFG,2004,1260.0605
3,Afghanistan,AFG,2005,1352.3207
4,Afghanistan,AFG,2006,1366.9932
...,...,...,...,...
6341,Zimbabwe,ZWE,2017,2331.7808
6342,Zimbabwe,ZWE,2018,2399.6216
6343,Zimbabwe,ZWE,2019,2203.3967
6344,Zimbabwe,ZWE,2020,1990.3195


In [6]:
DATA_PATH = "./Energy_graph/data/metadata/"


# get GDP(PPP) for given country and year, the GDP is in 2017 USD 
def get_country_GDP(country : str, year: int):
    gpd_PPP =  pd.read_csv(DATA_PATH +"Wages/gdp-per-capita-worldbank.csv")
    df = gpd_PPP[gpd_PPP["Year"] == year].drop(columns=["Year", "Entity"])

    
    df.reset_index(inplace=True, drop=True)
    df.set_index("Code", inplace=True)

    country_code = pycountry.countries.search_fuzzy(country)[0].alpha_3

    return df.loc[country_code].values[0]


get_country_GDP("United States", 2020)

60167.855

In [24]:
df.loc[res.json()['address']['country']].values[0]

46064.254

# Average wages
https://data.oecd.org/earnwage/average-wages.htm

In [10]:

wages = pd.read_csv("./Energy_graph/data/metadata/Wages/average_wages_OECD.csv")
# average wages in 2021 in the OECD countries
wages_processed = wages.drop(columns=[ "INDICATOR", "SUBJECT", "MEASURE", "FREQUENCY", "Flag Codes"]).set_index(["LOCATION", "TIME"])
wages_processed

Value
LOCATION TIME              
AUS      1991  42309.381476
         1992  43173.771334
         1993  43578.241422
         1994  43873.856945
         1995  43715.000000
...                     ...
CRI      2017  27020.000000
         2018  27709.000000
         2019  28524.000000
         2020  31341.000000
         2021  31117.834263

[1173 rows x 1 columns]

# Population density
https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-rev11

In [20]:
DATA_PATH = "./Energy_graph/data/metadata/"

latitude =46.056946
longitude = 14.505751


def get_population_density(latitude : float, longitude: float) -> float:
        
    # Open the dataset
    with rasterio.open(DATA_PATH+ "Population/Density_5min/gpw_v4_population_density_rev11_2020_2pt5_min.asc") as ds:
    
        # Convert your latitude and longitude to dataset's coordinate system
        x, y = transform('EPSG:4326', ds.crs, [longitude], [latitude])

        # Fetch the population density value at the given coordinates
        row, col = ds.index(x[0], y[0])
        population_density = ds.read(1)[row, col]

    return population_density

get_population_density(latitude, longitude)




1897.517

## Elevation and Topography
https://open-meteo.com/en/docs/elevation-api

In [21]:
lat = 52.52
lon = 13.41


def get_elevation(lat : float, lon: float) -> float:
    # get elevation from open-meteo api
    elevation = requests.get(f"https://api.open-meteo.com/v1/elevation?latitude={lat}&longitude={lon}")
    
    return elevation.json()["elevation"][0]

get_elevation(27.986065, 86.922623)

8395.0

# Education
https://data.oecd.org/eduatt/adult-education-level.htm

In [22]:
path = "./Energy_graph/data/metadata/Population/Education/adult_education_levels.csv"

df = pd.read_csv(path)
df[df["TIME"] == 2007].drop(columns=["TIME", "Flag Codes", "MEASURE", "FREQUENCY", "INDICATOR"]).set_index("LOCATION")


,SUBJECT,Value
LOCATION,,
AUS,BUPPSRY,31.816732
AUS,TRY,33.736710
AUS,UPPSRY,34.446560
AUT,BUPPSRY,20.062889
AUT,TRY,25.092672
...,...,...
LTU,TRY,28.931429
LTU,UPPSRY,59.999691
CRI,BUPPSRY,62.960529


In [23]:

def get_country_code(country):
    return pycountry.countries.search_fuzzy(country)[0].alpha_3


def get_education_level(country: str, year: int) -> float:
    df = pd.read_csv(DATA_PATH + "Population/Education/adult_education_levels.csv")
    if year not in df["TIME"].unique():
        raise ValueError("Year must be between 2018 and 2021")
    
    # drop unnecessary columns and set index to country code
    df = df[df["TIME"] == year].drop(columns=["TIME", "Flag Codes", "MEASURE", "FREQUENCY", "INDICATOR"]).set_index("LOCATION")

    # get ISO-3166 alpha-3 country code from country name
    country_code = get_country_code(country)

    # ages 25-64
    # % of population with below upper secondary education, % of population with upper secondary, % of population with tertiary education
    return df.loc[country_code].values[0][1],df.loc[country_code].values[2][1],  df.loc[country_code].values[1][1]

get_education_level("Australia", 2007)
    

(31.816732, 34.44656, 33.73671)

# Energy sources


In [13]:
# https://data.oecd.org/

path = "./Energy_graph/data/metadata/Energy/DP_LIVE_25072023101159292.csv"
df = pd.read_csv(path)
df

# could potentialy be useful

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1960,31.483,NaN
1,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1961,32.664,NaN
2,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1962,34.085,NaN
3,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1963,35.961,NaN
4,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1964,37.404,NaN
...,...,...,...,...,...,...,...,...
18471,UGA,PRYENRGSUPPLY,TOT,TOE_1000USD,A,2017,0.175,NaN
18472,UGA,PRYENRGSUPPLY,TOT,TOE_1000USD,A,2018,0.176,NaN
18473,UGA,PRYENRGSUPPLY,TOT,TOE_1000USD,A,2019,0.178,NaN
18474,UGA,PRYENRGSUPPLY,TOT,TOE_1000USD,A,2020,0.172,NaN


# Walk score US&CANADA only


In [ ]:
# https://www.walkscore.com/professional/api.php

# Energy prices EEA
Gas:
https://ec.europa.eu/eurostat/databrowser/view/NRG_PC_202__custom_7003504/default/table?lang=en

Electricity:
https://ec.europa.eu/eurostat/databrowser/view/NRG_PC_204__custom_7006505/default/table?lang=en

In [28]:

def get_semester(date: date):
    if date.month < 7:
        return str(date.year) + "-S1"
    else:
        return str(date.year) + "-S2"
    
path ="./Energy_graph/data/metadata/Energy/Gas_prices_EEA.csv"
df = pd.read_csv(path)
df = df.drop(columns=["DATAFLOW", "LAST UPDATE", "freq", "product", "nrg_cons", "unit", "tax", "currency", "OBS_FLAG"])
# set index to country code and year
df = df.set_index(["geo", "TIME_PERIOD"])

df.loc["AT", "2021-S2"].values[0]
# df[df["nrg_cons"] != "GJ20-199"]

0.0602

In [29]:
from datetime import date
# gas prices in PPS in kWh for medium sized households 
def get_gas_price(country: str, date: date) -> float:
    if date.year < 2007 or date.year > 2022:
        raise ValueError("Year must be between 2007 and 2022")
    
    df = pd.read_csv(DATA_PATH + "Energy/Gas_prices_EEA.csv")
    df= df.drop(columns=["DATAFLOW", "LAST UPDATE", "freq", "product", "nrg_cons", "unit", "tax", "currency", "OBS_FLAG"])
    df = df.set_index(["geo", "TIME_PERIOD"])

    country_code = pycountry.countries.search_fuzzy(country)[0].alpha_2
    semester = get_semester(date)

    return df.loc[country_code, semester].values[0]
    

get_gas_price("Germany", date(2019, 7, 1))


0.0532

In [63]:


# electricity prices in PPS per kWh for given country and date(bi-annual data only)
def get_electricity_price(country: str, date: date) -> float:
    if date.year < 2007 or date.year > 2022:
        raise ValueError("Year must be between 2007 and 2022")
    
    df = pd.read_csv(DATA_PATH + "Energy/Electricity_prices_EEA.csv")
    df.drop(columns=["LAST UPDATE", "freq", "product","OBS_FLAG", "DATAFLOW", "currency", "tax", "unit"])
    # average electricity prices in PPS per kWh for diffrent yearly consumption levels
    df_grouped = df.groupby(['geo', 'TIME_PERIOD'])['OBS_VALUE'].mean()

    country_code = pycountry.countries.search_fuzzy(country)[0].alpha_2
    semester = get_semester(date)
    
    return df_grouped.loc[country_code, semester]

get_electricity_price("Germany", date(2019, 7, 7))

0.26426666666666665

# Cooling and heating degree days by country

https://ec.europa.eu/eurostat/databrowser/view/NRG_CHDD_A/default/table?lang=en

explanation:https://ec.europa.eu/eurostat/cache/metadata/en/nrg_chdd_esms.htm

In [31]:

path = "./Energy_graph/data/metadata/Energy/Heating_cooling_index.csv"
df = pd.read_csv(path)

# drop useless columns
df.drop(columns=["DATAFLOW", "LAST UPDATE","unit", "freq", "OBS_FLAG"], inplace=True)


# Pivot the DataFrame to reshape it. 
# The unique values in column 'indic_nrg' become the new columns in the output DataFrame.
df_pivot = df.pivot(index=['geo', 'TIME_PERIOD'], columns='indic_nrg', values='OBS_VALUE')
df_pivot

indic_nrg          CDD      HDD
geo TIME_PERIOD                
AT  2007         25.74  3363.28
    2008          7.30  3444.43
    2009          9.94  3504.84
    2010         25.27  3905.88
    2011         17.63  3409.28
...                ...      ...
SK  2018         53.06  2923.83
    2019         65.09  2899.05
    2020         19.95  3046.66
    2021         62.28  3382.63
    2022         67.77  3042.68

[464 rows x 2 columns]

In [32]:

def get_cooling_and_heating_degree_days(country : str, year: int) -> float:
    df = pd.read_csv(DATA_PATH + "Energy/Heating_cooling_index.csv")

    # drop useless columns
    df.drop(columns=["DATAFLOW", "LAST UPDATE","unit", "freq", "OBS_FLAG"], inplace=True)
    df_pivot = df.pivot(index=['geo', 'TIME_PERIOD'], columns='indic_nrg', values='OBS_VALUE')
    
    country_code = pycountry.countries.search_fuzzy(country)[0].alpha_2
    

    return df_pivot.loc[country_code, year]["CDD"], df_pivot.loc[country_code, year]["HDD"]

get_cooling_and_heating_degree_days("Germany", 2019)


(46.05, 2800.81)

# Public holidays
https://pypi.org/project/holidays/

In [12]:



from datetime import date

import holidays

def get_holidays(country, year):
    """Returns a list of holidays for a given country and year"""
    country = pc.country_name_to_country_alpha2(country)
    try:
        # Getting the country's holiday calendar
        country_holidays = getattr(holidays, country)(years=year)
        # Printing all holidays for that year
        dates = []
        for date, name in sorted(country_holidays.items()):
            dates.append(date)

        return dates
    except AttributeError:
        return None

# Example usage
get_holidays('DE', 2023)


[datetime.date(2023, 1, 1),
 datetime.date(2023, 4, 7),
 datetime.date(2023, 4, 10),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 5, 18),
 datetime.date(2023, 5, 29),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 12, 25),
 datetime.date(2023, 12, 26)]

In [10]:
dict(us_holidays)

{}

In [9]:
def country_to_continent(country_name):
    country_code = pc.country_name_to_country_alpha2(country_name)
    continent_name = pc.country_alpha2_to_continent_code(country_code)

    continent_dict = {
    'AF': 'Africa',
    'AS': 'Asia',
    'EU': 'Europe',
    'NA': 'North America',
    'SA': 'South America',
    'OC': 'Oceania',
    'AN': 'Antarctica',
    }

    return continent_dict[continent_name]

country_to_continent('New Zealand')

'Oceania'

In [10]:

def search_country(country_name):
    url = f"https://nominatim.openstreetmap.org/search?country={country_name}&format=json"

    response = requests.get(url)
    data = response.json()

    return data


search_country("Germany")



[{'place_id': 307772656,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 51477,
  'boundingbox': ['47.2701114', '55.099161', '5.8663153', '15.0419309'],
  'lat': '51.1638175',
  'lon': '10.4478313',
  'display_name': 'Deutschland',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.8896814891722549,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}]

In [26]:
%pip install tzwhere


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.0 MB/s eta 0:00:00
  Created wheel for tzwhere: filename=tzwhere-3.0.3-py3-none-any.whl size=23742580 sha256=e0d26e3fcdd1d0273efdde3309d8a3b0bdc0cfa8298ed761e046acb06994542a
  Stored in directory: /home/jovyan/shared/anaconda/pip/wheels/a0/db/3b/5a63380a36e6a302b8aef9418997be589256d183179a25a227
Successfully built tzwhere
Note: you may need to restart the kernel to use updated packages.


In [27]:
from tzwhere.tzwhere import tzwhere

def get_timezone_from_coordinates(lat, lon):
    tz_str = tzwhere().tzNameAt(lat, lon)
    return tz_str

latitude = 51.5074  # latitude for London, UK
longitude = -0.1278  # longitude for London, UK
print(get_timezone_from_coordinates(latitude, longitude))  # Prints: 'Europe/London'

/opt/conda/lib/python3.10/site-packages/tzwhere/tzwhere.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.timezoneNamesToPolygons[tzname] = WRAP(polys)


Europe/London
